In [0]:
# imports

import os
import requests
import json
import ollama
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [0]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

# System prompt for the AI TECHNICAL LLM AND PYTHON TUTOR."

system_prompt = "You are an EXPERT in AI, LLMS and Python \
Provide the answer with example ALLWAYS when necessary. \
If you do not know the answer just say 'I don't know the answer' \
Respond in markdown in Spanish."

# messages
def messages_for(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
question = question[:5_000] # Truncate if more than 5,000 characters

In [0]:
# METHODS TO DISPLAY
def display_summary_ollama(question):
    response = ollama.chat(
        model = MODEL_LLAMA,
        messages = messages_for(question)
    )    
    summary = response['message']['content']
    display(Markdown(summary))

def display_summary_gpt(question):
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = messages_for(question),
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    
def display_summary(llm, question):
    if llm.startswith("llama3.2"):
        display_summary_ollama(question)
    else:
        display_summary_gpt(question)

In [0]:
# Get gpt-4o-mini to answer, with streaming

display_summary(MODEL_GPT, question)

In [0]:
# Get Llama 3.2 to answer

display_summary(MODEL_LLAMA, question)